# Chapter 0: Fundamentals

## 0.1Evaluationg Polynomials

Consider the polynomial $P(x) = 2x^4 + 3x^3 -3x^2 +5x -1 $. Its value at $x = \frac{1}{2}$ can be calculated using:

In [ ]:
%%timeit
x= 1/2
P = 2*x*x*x*x + 3*x*x*x - 3*x*x + 5*x -1

403 ns ± 8.98 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


A faster method could be calculating the powers of $x$ and storing them:

In [ ]:
|%%timeit
x= 1/2
x2 = x*x
x3 = x2*x
x4 = x3*x
P = 2*x4 + 3*x3 -3*x2 + 5*x -1

However, using nested multiplication or __Horner's method__ is faster than the methods above:

In [ ]:
%%timeit
x= 1/2
P = -1 + x*(5+x*(-3+x*(3+x*2)))

314 ns ± 84.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


A generalized function to calculate polynomials using Horner's method is:

In [ ]:
import numpy as np

def nest(d, c, x, b=None):
    if b is None:
        b = np.zeros(d)
    y = c[d]
    for i in range(d, 0, -1):
        y = y * (x - b[i-1]) + c[i-1]
    return y


In order to calculate the previously given polynomial we simply use the following instruction:

In [ ]:
%%timeit
result = nest(4, [-1, 5, -3, 3, 2], 1/2, [0, 0, 0, 0])

## 0.3 Floating Point Representation

IEEE 754 is a technical standard for floating point arithmetic. In IEEE format, real numbers are represented as:

$$ \pm 1.bbbbb \times 2^p $$

Almost all platforms map python floats to IEEE 754 binary64. According to IEEE 754 standard, a binary64 consists of:
- sign bit: 0 for positive numbers and 1 for negative numbers
- Exponent: 11 bits representing a value ranging from -1022 to 1023 (actual value stored is between 1 and 2046, bias of 1023 is subtracted to obtain the value of the exponent. 0 and 2047 are reserved for subnormal numbers, $\infty$ and `NaN`).  
- Mantissa: 52 bits to store 53 significant digits (leading digit is not stored because it is always 1).

Most fractions (and decimals) cannot be represented exactly as a binary fractions. For example the decimal 0.1 is actually:

In [ ]:
format(0.1, '.20f')

In python, hexadecimal representation of a float can be displayed using the built-in `float.hex(y)` function. For the number 1.0 the float representation is:

In [ ]:
print(float.hex(1.0))

Note that the next number that can be represented using a binary64 is:

In [ ]:
x = float.fromhex('0x1.0000000000001p+0')
print(x)

Which is equal to $1 + 2^{-52}$

In [ ]:
print((x-1).hex())

The difference between 1 and the smallest number greater than 1 is called __machine epsilon__, denoted as $\epsilon_{mach}$. Note that $\frac{1}{2} \epsilon_{mach}$ is the largest possible rounding error for an arithmetic operation.

## 0.4 Loss of Significance

Let's consider the function $E_1 = \frac{1 - \cos x}{\sin^2 x} $. At $x = 0$ both the numerator and denominator are zero and the function is undefined. Plotting the function reveals that value of $E_1$ should approach $\frac {1}{2}$ as $x$ appoaches $0$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the function
def f(x):
    return (1 - np.cos(x)) / np.sin(x)**2

# Generate x values
x_values = np.linspace(-1, 1, 1000)

# Exclude points where the function is undefined
x_values = x_values[x_values % np.pi != 0]

# Plot the function
plt.plot(x_values, f(x_values), label=r'$\frac{1 - \cos x}{\sin^2 x}$')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Plot of the function $\\frac{1 - \\cos x}{\\sin^2 x}$')
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.axvline(0, color='black', linewidth=0.5, linestyle='--')
plt.legend()
plt.grid(True)
plt.show()

However, computing value of $E_1$ for a very small $x$ e.g. $10^{-10}$ gives the following:

In [ ]:
import numpy as np
x = 10**-10
E1 = (1 - np.cos(x))/(np.sin(x)**2)
print(E1)

This is because $\cos x$ is very close to 1, the numerator is calculated as $0$; while the denominator still has a greater than zero value. To get around this issue, we multiply both the numerator and denominator by $1+\cos x$ and obtain:
$$\frac{(1-\cos x)(1+\cos x)}{(\sin^2 x)(1+\cos x)} = \frac{1-\cos^2 x}{(\sin^2 x)(1+\cos x)} $$
Using the identity $\sin^2 x + \cos^2 x = 1$ the numerator becomes $\sin^2 x$ and we get:
$$\frac{\sin^2 x}{(\sin^2 x)(1+\cos x)} = \frac{1}{1+\cos x} $$
Let's call this $E_2$ and calculate the values of $E_1$ and $E_2$ for different values of $x$:

In [ ]:
import numpy as np

# Generate x values
x_values = np.geomspace(1, 1e-14, 15)

# Create a table
print("|         x         |        E1         |        E2        |       cos x     |")
print("|-------------------|-------------------|------------------|-----------------|")

for x in x_values:
    E1 = (1 - np.cos(x))/(np.sin(x)**2)
    E2 = 1 / (1 + np.cos(x))
    E3 = np.cos(x)
    print(f"| {x:.15f} | {E1:.15f} |{E2:.15f} |{E3:.15f}| ")